<a href="https://colab.research.google.com/github/ru2zi/Book_Genre_Classification/blob/main/model_score_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip uninstall sentencepiece -y
!pip install sentencepiece

Found existing installation: sentencepiece 0.1.99
Uninstalling sentencepiece-0.1.99:
  Successfully uninstalled sentencepiece-0.1.99
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorWithPadding, TrainingArguments
import numpy as np
from torch.utils.data import Dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from transformers import AutoModelForSequenceClassification, Trainer, AutoTokenizer
from tqdm import tqdm
from datasets import Dataset
import pandas as pd
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from torch.optim import AdamW
import numpy as np
import torch.nn as nn
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gc.collect()

60

In [ ]:
# model_name = "microsoft/deberta-v3-base"
model_names = [
    'valurank/finetuned-distilbert-news-article-categorization',
    'sileod/deberta-v3-base-tasksource-nli',
    'andi611/distilbert-base-uncased-ner-agnews',
    'joeddav/distilbert-base-uncased-agnews-student',
    'facebook/bart-base'
]

model_names = model_names[4:]
model_names

['facebook/bart-base']

In [ ]:


model_names = ['sentence-transformers/all-mpnet-base-v2']
model_names

['sentence-transformers/all-mpnet-base-v2']

In [ ]:
data

,Title,label_encode,fold
0,The Oz Family Kitchen: More Than 100 Simple an...,5,0.0
1,Living with Intent: My Somewhat Messy Journey ...,20,3.0
2,Redshirts: A Novel with Three Codas,18,1.0
3,To Kill a Mockingbird,14,3.0
4,Canning and Preserving of Food Products with B...,5,0.0
...,...,...,...
66831,"Casenote Legal Briefs: Family Law, Keyed to Ha...",11,0.0
66832,Communication for Families in Crisis: Theories...,11,2.0
66833,"Fine-line Lithography (Materials processing, t...",0,3.0
66834,Magic Eye 2016 Wall Calendar,2,3.0


In [ ]:
datadata = pd.read_csv('/content/drive/MyDrive/Dacon/책분류/data/train_data_total.csv', encoding='latin-1')

data = data[['Title', 'label_encode']]

# Stratify 폴드
kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
for f, (t_, v_) in enumerate(kf.split(X=data.Title, y=data.label_encode)):
    data.loc[v_, 'fold'] = f

data = data[:]

train_df = data[data.fold!=0]
valid_df = data[data.fold==0]

train_dataset = Dataset.from_pandas(train_df, split="train")
valid_dataset = Dataset.from_pandas(valid_df, split="test")

In [ ]:
def preprocess_function(examples, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return tokenizer(examples['Title'], max_length=256, padding='max_length', truncation=True)

def get_dataloader(train_dataset, valid_dataset, model_name):
    train_data = train_dataset.map(lambda examples: preprocess_function(examples, model_name), batched=True)
    val_data = valid_dataset.map(lambda examples: preprocess_function(examples, model_name), batched=True)

    train_data = train_data.remove_columns(["fold"])
    train_data = train_data.remove_columns(['Title'])
    train_data = train_data.remove_columns(["__index_level_0__"])

    val_data = val_data.remove_columns(["fold"])
    val_data = val_data.remove_columns(['Title'])
    val_data = val_data.remove_columns(["__index_level_0__"])

    train_data = train_data.rename_column("label_encode", "labels")
    val_data = val_data.rename_column("label_encode", "labels")

    train_data.set_format("torch")
    val_data.set_format("torch")

    train_dataloader = DataLoader(train_data, shuffle=True, batch_size=8)
    eval_dataloader = DataLoader(val_data, batch_size=8)

    return train_dataloader, eval_dataloader

In [ ]:
def CrossEntropyLoss(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)

# Step 7: Training and validation
def train(model, data_loader, optimizer, device):
    model.train()
    losses = []

    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits

        loss = CrossEntropyLoss(logits, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Train Loss: {np.mean(losses)}")

def eval(model, data_loader, device):
  model.eval()
  y_true, y_pred = [], []
  losses = []
  with torch.no_grad():
    for batch in tqdm(data_loader):
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      outputs = model(input_ids, attention_mask=attention_mask)

      logits = outputs.logits
      loss = CrossEntropyLoss(logits, labels)

      losses.append(loss.item())
      predictions = torch.argmax(outputs.logits, dim=1)
      y_true.extend(labels.cpu().numpy())
      y_pred.extend(predictions.cpu().numpy())
    print(f"Valid Loss: {np.mean(losses)}")
    gc.collect()

  return y_true, y_pred


In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)

    best_macro_f1 = 0
    best_model = None
    for epoch in range(10):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
            model_name_clean = model_name.replace('/','_')
            torch.save(model.state_dict(),
                       f'/content/drive/MyDrive/Dacon/책분류/best2/best_model_{round(best_macro_f1,4)}_{model_name_clean}.pt')

    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")



Map:   0%|          | 0/53468 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v2 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a

Epoch: 1


 37%|███▋      | 2441/6684 [14:47<25:42,  2.75it/s]


KeyboardInterrupt: ignored

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import f1_score
import pandas as pd

# Set the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model names and corresponding best.pt file paths
model_names = [
    ('valurank/finetuned-distilbert-news-article-categorization', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6697_joeddav_distilbert-base-uncased-agnews-student.pt'),
    ('sileod/deberta-v3-base-tasksource-nli', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6592_sileod deberta-v3-base-tasksource-nli.pt'),
    ('andi611/distilbert-base-uncased-ner-agnews', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6677_andi611_distilbert-base-uncased-ner-agnews.pt'),
    ('joeddav/distilbert-base-uncased-agnews-student', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6697_joeddav_distilbert-base-uncased-agnews-student.pt'),
    ('facebook/bart-base', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6663_facebook_bart-base.pt')
]

# Load the test data from test.csv
test_data = pd.read_csv('/content/drive/MyDrive/Dacon/책분류/test_data.csv')

# Example: Select specific columns from the test data
test_data = test_data[['Title']]
test_dataset = Dataset.from_pandas(test_data, split="test")

def preprocess_function(examples, tokenizer):
    return tokenizer(examples["Title"], max_length=256, padding='max_length', truncation=True)

# Define the get_test_dataloader function for test data
def get_test_dataloader(dataset, tokenizer):
    dataset = dataset.map(lambda examples: preprocess_function(examples, tokenizer), batched=True)
    dataset = dataset.remove_columns(["Title"])
    dataset.set_format("torch")
    dataloader = DataLoader(dataset, batch_size=8)
    return dataloader

# Load the best models and make predictions
all_predictions = []
for model_name, model_path in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    test_dataloader = get_test_dataloader(test_dataset, tokenizer)
    predictions = []
    with torch.no_grad():
        for batch in test_dataloader:
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)

          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)

          outputs = model(input_ids, attention_mask=attention_mask)
          logits = outputs.logits
          probabilities = nn.functional.softmax(logits, dim=1)
          predicted_labels = torch.argmax(probabilities, dim=1).tolist()
          predictions.extend(predicted_labels)

    all_predictions.append(predictions)

# Take the majority vote to determine the final predictions
final_predictions = []
for i in range(len(test_data)):
    labels = [pred[i] for pred in all_predictions]
    print(labels)
    majority_vote = max(set(labels), key=labels.count)
    final_predictions.append(majority_vote)

test_data = pd.read_csv('/content/drive/MyDrive/Dacon/책분류/test_data.csv')
# Create a submission DataFrame
submission_df = pd.DataFrame({'id': test_data['id'], 'label': final_predictions})
label_mapping = {label: i for i, label in enumerate(data['label'].unique())}
label2id = {label: i for i, label in enumerate(label_mapping)}
id2label = {v:k for k,v in label2id.items()}
submission_df['label'] = submission_df['label'].map(id2label)

# Save the submission DataFrame to a CSV file
submission_df.to_csv('/content/drive/MyDrive/Dacon/책분류/data/best_pt/submission.csv', index=False)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at valurank/finetuned-distilbert-news-article-categorization and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([8, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([8]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/29436 [00:00<?, ? examples/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at sileod/deberta-v3-base-tasksource-nli and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/29436 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at andi611/distilbert-base-uncased-ner-agnews and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at joeddav/distilbert-base-uncased-agnews-student and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/29436 [00:00<?, ? examples/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight', 'classification_head.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/29436 [00:00<?, ? examples/s]

KeyError: ignored

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import f1_score
import pandas as pd
from collections import Counter

# Set the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model names and corresponding best.pt file paths
model_names = [
    ('valurank/finetuned-distilbert-news-article-categorization', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6697_joeddav_distilbert-base-uncased-agnews-student.pt'),
    ('sileod/deberta-v3-base-tasksource-nli', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6592_sileod deberta-v3-base-tasksource-nli.pt'),
    ('andi611/distilbert-base-uncased-ner-agnews', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6677_andi611_distilbert-base-uncased-ner-agnews.pt'),
    ('joeddav/distilbert-base-uncased-agnews-student', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6697_joeddav_distilbert-base-uncased-agnews-student.pt'),
    ('facebook/bart-base', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6663_facebook_bart-base.pt')
]

# Load the test data from test.csv
test_data = pd.read_csv('/content/drive/MyDrive/Dacon/책분류/test_data.csv')

# Example: Select specific columns from the test data
test_data = test_data[['Title']]
test_dataset = Dataset.from_pandas(test_data, split="test")

def preprocess_function(examples, tokenizer):
    return tokenizer(examples["Title"], max_length=256, padding='max_length', truncation=True)

# Define the get_test_dataloader function for test data
def get_test_dataloader(dataset, tokenizer):
    dataset = dataset.map(lambda examples: preprocess_function(examples, tokenizer), batched=True)
    dataset = dataset.remove_columns(["Title"])
    dataset.set_format("torch")
    dataloader = DataLoader(dataset, batch_size=8)
    return dataloader

# Load the best models and make predictions
all_predictions = []
all_probabilities = []  # Track probabilities for each model
for model_name, model_path in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    test_dataloader = get_test_dataloader(test_dataset, tokenizer)
    predictions = []
    probabilities = []
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            batch_probabilities = nn.functional.softmax(logits, dim=1)
            batch_predictions = torch.argmax(batch_probabilities, dim=1).tolist()
            probabilities.extend(batch_probabilities.tolist())
            predictions.extend(batch_predictions)

    all_predictions.append(predictions)
    all_probabilities.append(probabilities)

# Take the majority vote to determine the final predictions
final_predictions = []
for i in range(len(test_data)):
    labels = [pred[i] for pred in all_predictions]
    label_counts = Counter(labels)
    majority_vote = label_counts.most_common(1)[0][0]
    final_predictions.append(majority_vote)

# Apply probability-based weighting
weighted_predictions = []
for i in range(len(test_data)):
    label_probs = [probs[i] for probs in all_probabilities]
    label_weights = [sum(prob[label] for prob in label_probs) for label in range(24)]
    weighted_vote = label_weights.index(max(label_weights))
    weighted_predictions.append(weighted_vote)

test_data = pd.read_csv('/content/drive/MyDrive/Dacon/책분류/test_data.csv')
# Create a submission DataFrame
submission_df = pd.DataFrame({'id': test_data['id'], 'label': weighted_predictions})
label_mapping = {label: i for i, label in enumerate(data['label'].unique())}
label2id = {label: i for i, label in enumerate(label_mapping)}
id2label = {v: k for k, v in label2id.items()}
submission_df['label'] = submission_df['label'].map(id2label)

# Save the submission DataFrame to a CSV file
submission_df.to_csv('/content/drive/MyDrive/Dacon/책분류/data/best_pt/submission.csv', index=False)



In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import f1_score
import pandas as pd
from collections import Counter

# Set the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model names and corresponding best.pt file paths
model_names = [
    ('valurank/finetuned-distilbert-news-article-categorization', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6697_joeddav_distilbert-base-uncased-agnews-student.pt'),
    ('sileod/deberta-v3-base-tasksource-nli', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6592_sileod deberta-v3-base-tasksource-nli.pt'),
    ('andi611/distilbert-base-uncased-ner-agnews', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6677_andi611_distilbert-base-uncased-ner-agnews.pt'),
    ('joeddav/distilbert-base-uncased-agnews-student', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6697_joeddav_distilbert-base-uncased-agnews-student.pt'),
    ('facebook/bart-base', '/content/drive/MyDrive/Dacon/책분류/data/best_pt/best_model_0.6663_facebook_bart-base.pt')
]


# Load the test data from test.csv
test_data = pd.read_csv('test.csv')

# Example: Select specific columns from the test data
test_data = test_data[['text']]

# Define the get_test_dataloader function for test data
def get_test_dataloader(dataset, tokenizer, max_len):
    encoded_inputs = tokenizer(
        dataset['text'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    dataset = torch.utils.data.TensorDataset(encoded_inputs['input_ids'], encoded_inputs['attention_mask'])
    dataloader = DataLoader(dataset, batch_size=8)
    return dataloader

# Load the best models and make predictions
all_predictions = []
all_probabilities = []  # Track probabilities for each model
for model_name, model_dir in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=8, ignore_mismatched_sizes=True)
    model.to(device)
    model.eval()

    test_dataloader = get_test_dataloader(test_data, tokenizer, max_len=512)
    predictions = []
    probabilities = []
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            batch_probabilities = nn.functional.softmax(logits, dim=1)
            batch_predictions = torch.argmax(batch_probabilities, dim=1).tolist()
            probabilities.extend(batch_probabilities.tolist())
            predictions.extend(batch_predictions)

    all_predictions.append(predictions)
    all_probabilities.append(probabilities)

# Take the majority vote to determine the final predictions
final_predictions = []
for i in range(len(test_data)):
    labels = [pred[i] for pred in all_predictions]
    label_counts = Counter(labels)
    majority_vote = label_counts.most_common(1)[0][0]
    final_predictions.append(majority_vote)

# Apply probability-based weighting
weighted_predictions = []
for i in range(len(test_data)):
    label_probs = [probs[i] for probs in all_probabilities]
    label_weights = [sum(prob[label] for prob in label_probs) for label in range(8)]
    weighted_vote = label_weights.index(max(label_weights))
    weighted_predictions.append(weighted_vote)

test_data = pd.read_csv('/content/drive/MyDrive/Dacon/책분류/test_data.csv')
# Create a submission DataFrame
submission_df = pd.DataFrame({'id': test_data['id'], 'label': weighted_predictions})
label_mapping = {label: i for i, label in enumerate(data['label'].unique())}
label2id = {label: i for i, label in enumerate(label_mapping)}
id2label = {v: k for k, v in label2id.items()}
submission_df['label'] = submission_df['label'].map(id2label)

# Save the submission DataFrame to a CSV file
submission_df.to_csv('/content/drive/MyDrive/Dacon/책분류/data/best_pt/submission.csv', index=False)


#['facebook/bart-base']
## Best F1 score: 0.6269216385763327

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:34<00:00,  8.66it/s]


...

Valid Loss: 1.5034736541906992
Val Macro F1: 0.6248042483261096
Epoch: 5


100%|██████████| 300/300 [00:34<00:00,  8.73it/s]


Train Loss: 0.40328566714500386


100%|██████████| 75/75 [00:02<00:00, 26.29it/s]


Valid Loss: 1.5422454329331716
Val Macro F1: 0.6269216385763327
Model Name: facebook/bart-base
Best F1 score: 0.6269216385763327


#['abhishek/autonlp-bbc-news-classification-37229289']

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at abhishek/autonlp-bbc-news-classification-37229289 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 1024]) in the checkpoint and torch.Size([24, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [01:25<00:00,  3.49it/s]


...

Valid Loss: 3.1308230463663738
Val Macro F1: 0.0033333333333333327
Epoch: 5


100%|██████████| 300/300 [01:25<00:00,  3.50it/s]


Train Loss: 3.1799859841664633


100%|██████████| 75/75 [00:07<00:00, 10.46it/s]


Valid Loss: 3.1503365230560303
Val Macro F1: 0.003715498938428875
Model Name: abhishek/autonlp-bbc-news-classification-37229289
Best F1 score: 0.003715498938428875


#['distilbert-base-uncased']

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

Epoch: 1


100%|██████████| 300/300 [00:14<00:00, 21.07it/s]


...

Valid Loss: 1.5575558571020762
Val Macro F1: 0.6166741653555379
Epoch: 5


100%|██████████| 300/300 [00:14<00:00, 21.09it/s]


Train Loss: 0.2002479961141944


100%|██████████| 75/75 [00:01<00:00, 61.74it/s]


Valid Loss: 1.692919378578663
Val Macro F1: 0.5985655199688641
Model Name: distilbert-base-uncased
Best F1 score: 0.5985655199688641


#['valurank/finetuned-distilbert-news-article-categorization']
## Best F1 score: 0.6226543386324948

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at valurank/finetuned-distilbert-news-article-categorization and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([8, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([8]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:14<00:00, 21.10it/s]


...

Valid Loss: 1.5603633618354797
Val Macro F1: 0.6070936213185506
Epoch: 5


100%|██████████| 300/300 [00:14<00:00, 21.14it/s]


Train Loss: 0.23160887466743588


100%|██████████| 75/75 [00:01<00:00, 61.51it/s]


Valid Loss: 1.6918715304136276
Val Macro F1: 0.6226543386324948
Model Name: valurank/finetuned-distilbert-news-article-categorization
Best F1 score: 0.6226543386324948


#['mosesju/distilbert-base-uncased-finetuned-news']

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at mosesju/distilbert-base-uncased-finetuned-news and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:14<00:00, 21.09it/s]


...

Valid Loss: 1.8307284756501516
Val Macro F1: 0.47050348596913577
Epoch: 5


100%|██████████| 300/300 [00:14<00:00, 21.12it/s]


Train Loss: 0.9076719348629315


100%|██████████| 75/75 [00:01<00:00, 61.62it/s]


Valid Loss: 1.9095929706096648
Val Macro F1: 0.49683330559654765
Model Name: mosesju/distilbert-base-uncased-finetuned-news
Best F1 score: 0.49683330559654765


#['fabriceyhc/bert-base-uncased-ag_news']

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at fabriceyhc/bert-base-uncased-ag_news and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:27<00:00, 10.98it/s]


...

Valid Loss: 1.6501842713356019
Val Macro F1: 0.5222619303723857
Epoch: 5


100%|██████████| 300/300 [00:27<00:00, 10.99it/s]


Train Loss: 0.884135784159104


100%|██████████| 75/75 [00:02<00:00, 31.95it/s]


Valid Loss: 1.6738179834683735
Val Macro F1: 0.5479566405274289
Model Name: fabriceyhc/bert-base-uncased-ag_news
Best F1 score: 0.5479566405274289


#['philschmid/distilbert-base-multilingual-cased-sentiment']
## Best F1 score: 0.5709860255859988

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at philschmid/distilbert-base-multilingual-cased-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:15<00:00, 19.12it/s]


...

Valid Loss: 1.756728039185206
Val Macro F1: 0.5602989731229068
Epoch: 5


100%|██████████| 300/300 [00:15<00:00, 19.16it/s]


Train Loss: 0.375507787677149


100%|██████████| 75/75 [00:01<00:00, 61.47it/s]


Valid Loss: 1.7420175896088281
Val Macro F1: 0.5709860255859988
Model Name: philschmid/distilbert-base-multilingual-cased-sentiment
Best F1 score: 0.5709860255859988


#['arjuntheprogrammer/distilbert-base-multilingual-cased-sentiment-2']

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at arjuntheprogrammer/distilbert-base-multilingual-cased-sentiment-2 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:15<00:00, 19.13it/s]


...

Valid Loss: 1.8099534447987875
Val Macro F1: 0.479768380084181
Epoch: 5


100%|██████████| 300/300 [00:15<00:00, 19.15it/s]


Train Loss: 0.6574703472604354


100%|██████████| 75/75 [00:01<00:00, 61.27it/s]


Valid Loss: 1.9037060475349425
Val Macro F1: 0.49883099413435383
Model Name: arjuntheprogrammer/distilbert-base-multilingual-cased-sentiment-2
Best F1 score: 0.49883099413435383


#['microsoft/deberta-v3-base']

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.out_proj.weight', 'classification_head.dense.bias', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:46<00:00,  6.43it/s]


...

Valid Loss: 1.506621154944102
Val Macro F1: 0.6152704039086137
Epoch: 5


100%|██████████| 300/300 [00:43<00:00,  6.82it/s]


Train Loss: 0.4500641112153729


100%|██████████| 75/75 [00:03<00:00, 21.29it/s]


Valid Loss: 1.6372829798857371
Val Macro F1: 0.6222723050968025
Model Name: facebook/bart-base
Best F1 score: 0.6222723050968025


#['sileod/deberta-v3-base-tasksource-nli']
##Best F1 score: 0.6169459657008173

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at sileod/deberta-v3-base-tasksource-nli and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:39<00:00,  7.60it/s]


...

Valid Loss: 1.6706882909933727
Val Macro F1: 0.5771333829339139
Epoch: 5


100%|██████████| 300/300 [00:39<00:00,  7.61it/s]


Train Loss: 0.704371069719394


100%|██████████| 75/75 [00:03<00:00, 23.73it/s]


Valid Loss: 1.6229191921154658
Val Macro F1: 0.6169459657008173
Model Name: sileod/deberta-v3-base-tasksource-nli
Best F1 score: 0.6169459657008173


#['mrm8488/bert-mini-finetuned-age_news-classification']
## Val Macro F1: 0.5904738349193172

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mrm8488/bert-mini-finetuned-age_news-classification and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 256]) in the checkpoint and torch.Size([24, 256]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:04<00:00, 60.82it/s]


...

Valid Loss: 1.4015617918968202
Val Macro F1: 0.5904738349193172
Epoch: 5


100%|██████████| 300/300 [00:04<00:00, 62.20it/s]


Train Loss: 0.8063332471251488


100%|██████████| 75/75 [00:00<00:00, 158.90it/s]


Valid Loss: 1.4318553698062897
Val Macro F1: 0.5783874642080512
Model Name: mrm8488/bert-mini-finetuned-age_news-classification
Best F1 score: 0.5783874642080512


#['JiaqiLee/bert-agnews']
##Best F1 score: 0.6055295902023462

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at JiaqiLee/bert-agnews and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:27<00:00, 10.98it/s]


...

Valid Loss: 1.5952258324623108
Val Macro F1: 0.5875580966851117
Epoch: 5


100%|██████████| 300/300 [00:27<00:00, 10.99it/s]


Train Loss: 0.2984929620847106


100%|██████████| 75/75 [00:02<00:00, 31.91it/s]


Valid Loss: 1.6314644080400467
Val Macro F1: 0.6055295902023462
Model Name: JiaqiLee/bert-agnews
Best F1 score: 0.6055295902023462


#['andi611/distilbert-base-uncased-ner-agnews']
##Best F1 score: 0.6141533746974991

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at andi611/distilbert-base-uncased-ner-agnews and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:14<00:00, 21.16it/s]


...

Valid Loss: 1.5986180766423543
Val Macro F1: 0.6008405604567321
Epoch: 5


100%|██████████| 300/300 [00:14<00:00, 21.17it/s]


Train Loss: 0.20807110608865817


100%|██████████| 75/75 [00:01<00:00, 61.62it/s]


Valid Loss: 1.6996877626578013
Val Macro F1: 0.6141533746974991
Model Name: andi611/distilbert-base-uncased-ner-agnews
Best F1 score: 0.6141533746974991


#['lucasresck/bert-base-cased-ag-news']
##Best F1 score: 0.5834945860157417

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at lucasresck/bert-base-cased-ag-news and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


...

Valid Loss: 1.573275510072708
Val Macro F1: 0.6050063180008095
Epoch: 5


100%|██████████| 300/300 [00:35<00:00,  8.38it/s]


Train Loss: 0.1860089452813069


100%|██████████| 75/75 [00:02<00:00, 31.78it/s]


Valid Loss: 1.7778853209813437
Val Macro F1: 0.5834945860157417
Model Name: lucasresck/bert-base-cased-ag-news
Best F1 score: 0.5834945860157417


#['nateraw/bert-base-uncased-ag-news']
##Best F1 score: 0.6057691134634403

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nateraw/bert-base-uncased-ag-news and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:27<00:00, 10.99it/s]


...

Valid Loss: 1.5592368259032567
Val Macro F1: 0.5943026502351682
Epoch: 5


100%|██████████| 300/300 [00:27<00:00, 10.99it/s]


Train Loss: 0.2656827656614284


100%|██████████| 75/75 [00:02<00:00, 31.84it/s]


Valid Loss: 1.6601222189267477
Val Macro F1: 0.6057691134634403
Model Name: nateraw/bert-base-uncased-ag-news
Best F1 score: 0.6057691134634403


#['joeddav/distilbert-base-uncased-agnews-student']
## Best F1 score: 0.614980489180879

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at joeddav/distilbert-base-uncased-agnews-student and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:14<00:00, 21.10it/s]


...

Valid Loss: 1.602442625761032
Val Macro F1: 0.5743912539188475
Epoch: 5


100%|██████████| 300/300 [00:14<00:00, 21.13it/s]


Train Loss: 0.23359084829688073


100%|██████████| 75/75 [00:01<00:00, 61.62it/s]


Valid Loss: 1.7255241632461549
Val Macro F1: 0.614980489180879
Model Name: joeddav/distilbert-base-uncased-agnews-student
Best F1 score: 0.614980489180879


# ['bertugmirasyedi/deberta-v3-base-book-classification']
##Best F1 score: 0.5762519252110004

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at bertugmirasyedi/deberta-v3-base-book-classification and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:39<00:00,  7.60it/s]


...

Valid Loss: 1.491182038784027
Val Macro F1: 0.5807470665964467
Epoch: 5


100%|██████████| 300/300 [00:39<00:00,  7.61it/s]


Train Loss: 0.7046220517406862


100%|██████████| 75/75 [00:03<00:00, 23.77it/s]


Valid Loss: 1.653514496088028
Val Macro F1: 0.5762519252110004
Model Name: bertugmirasyedi/deberta-v3-base-book-classification
Best F1 score: 0.5762519252110004


#['sentence-transformers/all-mpnet-base-v2']
##Best F1 score: 0.6334328234417793

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v2 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a

Epoch: 1


100%|██████████| 300/300 [00:29<00:00, 10.26it/s]


...

Valid Loss: 1.5058498732248942
Val Macro F1: 0.6076338834878237
Epoch: 5


100%|██████████| 300/300 [00:29<00:00, 10.28it/s]


Train Loss: 0.45318602442741396


100%|██████████| 75/75 [00:02<00:00, 29.71it/s]


Valid Loss: 1.5184443378448487
Val Macro F1: 0.6334328234417793
Model Name: sentence-transformers/all-mpnet-base-v2
Best F1 score: 0.6334328234417793


#['sentence-transformers/all-MiniLM-L6-v2']
##Best F1 score: 0.5833636646839127

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:07<00:00, 42.59it/s]


...

Valid Loss: 1.3996585174401601
Val Macro F1: 0.5969995230700043
Epoch: 5


100%|██████████| 300/300 [00:07<00:00, 42.77it/s]


Train Loss: 0.47931636244058606


100%|██████████| 75/75 [00:00<00:00, 118.22it/s]


Valid Loss: 1.5091197196642558
Val Macro F1: 0.5833636646839127
Model Name: sentence-transformers/all-MiniLM-L6-v2
Best F1 score: 0.5833636646839127


#['cross-encoder/nli-deberta-base']
##Best F1 score: 0.569753616392932

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-deberta-base and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:39<00:00,  7.64it/s]


...

Valid Loss: 1.516977991660436
Val Macro F1: 0.5948431285241385
Epoch: 5


100%|██████████| 300/300 [00:39<00:00,  7.66it/s]


Train Loss: 0.4732176395008961


100%|██████████| 75/75 [00:03<00:00, 23.83it/s]


Valid Loss: 1.876776119073232
Val Macro F1: 0.569753616392932
Model Name: cross-encoder/nli-deberta-base
Best F1 score: 0.569753616392932


#['MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7']
##Best F1 score: 0.5411056544661026

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([24, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([24]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1


100%|██████████| 300/300 [00:41<00:00,  7.24it/s]


...

Valid Loss: 1.8087976721922556
Val Macro F1: 0.47271638984219244
Epoch: 5


100%|██████████| 300/300 [00:41<00:00,  7.24it/s]


Train Loss: 1.1661244600017866


100%|██████████| 75/75 [00:03<00:00, 23.69it/s]


Valid Loss: 1.7567442186673483
Val Macro F1: 0.5411056544661026
Model Name: MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7
Best F1 score: 0.5411056544661026


#['typeform/distilbert-base-uncased-mnli']
##Best F1 score: 0.5864732132799655

In [ ]:
for model_name in model_names:
    train_dataloader, eval_dataloader = get_dataloader(train_dataset, valid_dataset, model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=24, ignore_mismatched_sizes=True)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_macro_f1 = 0
    for epoch in range(5):
        print(f"Epoch: {epoch+1}")
        train(model, train_dataloader, optimizer, device)
        y_true_val, y_pred_val= eval(model, eval_dataloader, device)

        val_macro_f1 = f1_score(y_true_val, y_pred_val, average='macro')

        print(f"Val Macro F1: {val_macro_f1}")

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
    print(f"Model Name: {model_name}")
    print(f"Best F1 score: {val_macro_f1}")

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from you

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


...

Valid Loss: 1.5686596800883612
Val Macro F1: 0.5997987055536501
Epoch: 5


100%|██████████| 300/300 [00:14<00:00, 21.13it/s]


Train Loss: 0.2065847134341796


100%|██████████| 75/75 [00:01<00:00, 61.58it/s]


Valid Loss: 1.7697402648131053
Val Macro F1: 0.5864732132799655
Model Name: typeform/distilbert-base-uncased-mnli
Best F1 score: 0.5864732132799655
